# Python 装饰器
2022 09 18 日

### 从函数中返回函数

In [1]:
def hi():
    return "hi yasoob!"

def doSomethingBeforeHi(func):
    print("I am doing some boring work before executing hi()")
    print(func())

doSomethingBeforeHi(hi)

I am doing some boring work before executing hi()
hi yasoob!


### 将函数作为参数传给另一个函数

In [2]:
def hi():
    return "hi yasoob!"

def doSomethingBeforeHi(func):
    print("I am doing some boring work before executing hi()")
    print(func())

doSomethingBeforeHi(hi)

I am doing some boring work before executing hi()
hi yasoob!


### 第一个装饰器

In [4]:
def a_new_decorator(a_func):

    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")

        a_func()

        print("I am doing some boring work after executing a_func()")

    return wrapTheFunction

def a_function_requiring_decoration():
    print("I am the function which needs some decoration")

a_function_requiring_decoration()

print("========================================================================")

a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration)
#now a_function_requiring_decoration is wrapped by wrapTheFunction()

a_function_requiring_decoration()

I am the function which needs some decoration
I am doing some boring work before executing a_func()
I am the function which needs some decoration
I am doing some boring work after executing a_func()


In [8]:
@a_new_decorator
def a_function_requiring_decoration():
    """Hey you! Decorate me!"""
    print("I am the function which needs some decoration to "
          "remove my foul smell")

a_function_requiring_decoration()

I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after executing a_func()


In [18]:
### 问题
print(a_function_requiring_decoration)
print(a_function_requiring_decoration.__name__)

<function a_function_requiring_decoration at 0x13ea444c0>
a_function_requiring_decoration


输出的函数名为装饰器的函数名

### 改进的装饰器

In [19]:
from functools import wraps

def a_new_decorator(a_func):
    @wraps(a_func) ## 保留装饰前的函数名
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        a_func()
        print("I am doing some boring work after executing a_func()")
    return wrapTheFunction

@a_new_decorator
def a_function_requiring_decoration():
    """Hey yo! Decorate me!"""
    print("I am the function which needs some decoration to "
          "remove my foul smell")

print(a_function_requiring_decoration.__name__)

a_function_requiring_decoration


@wraps接受一个函数来进行装饰，并加入了复制函数名称、注释文档、参数列表等等的功能。这可以让我们在装饰器里面访问在装饰之前的函数的属性。

## 模版

In [20]:
from functools import wraps
def decorator_name(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated

@decorator_name
def func():
    return("Function is running")

can_run = True
print(func())

can_run = False
print(func())

Function is running
Function will not run


## 使用场景

### 授权

装饰器能有助于检查某个人是否被授权去使用一个web应用的端点(endpoint)。它们被大量使用于Flask和Django web框架中。这里是一个例子来使用基于装饰器的授权：

### 日志


In [23]:
from functools import wraps

def logit(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + " was called")
        result = func(*args, **kwargs)
        print(result) ## add print to result
        return result
    return with_logging

@logit
def addition_func(x,y):
   return x + y

result = addition_func(4,6)

addition_func was called


### 带参数的装饰器
来想想这个问题，难道@wraps不也是个装饰器吗？但是，它接收一个参数，就像任何普通的函数能做的那样。那么，为什么我们不也那样做呢？ 这是因为，当你使用@my_decorator语法时，你是在应用一个以单个函数作为参数的一个包裹函数。记住，Python里每个东西都是一个对象，而且这包括函数！记住了这些，我们可以编写一下能返回一个包裹函数的函数。

### 在函数中嵌入装饰器
我们回到日志的例子，并创建一个包裹函数，能让我们指定一个用于输出的日志文件。


In [24]:
from functools import wraps

def logit(logfile='out.log'):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # 打开logfile，并写入内容
            with open(logfile, 'a') as opened_file:
                # 现在将日志打到指定的logfile
                opened_file.write(log_string + '\n')
            return func(*args, **kwargs)
        return wrapped_function
    return logging_decorator

@logit()
def myfunc1():
    pass

myfunc1()
# 现在一个叫做 out.log 的文件出现了，里面的内容就是上面的字符串

@logit(logfile='func2.log')
def myfunc2():
    pass

myfunc2()
# 现在一个叫做 out.log 的文件出现了，里面的内容就是上面的字符串


myfunc1 was called
myfunc2 was called


## 装饰器类

现在我们有了能用于正式环境的logit装饰器，但当我们的应用的某些部分还比较脆弱时，异常也许是需要更紧急关注的事情。比方说有时你只想打日志到一个文件。而有时你想把引起你注意的问题发送到一个email，同时也保留日志，留个记录。这是一个使用继承的场景，但目前为止我们只看到过用来构建装饰器的函数。

幸运的是，类也可以用来构建装饰器。那我们现在以一个类而不是一个函数的方式，来重新构建logit。

In [33]:
class C(): # python3 默认继承 object 可以不写
    def __init__(self,cc = 20):
        self.cc = cc
    def __call__(self):
        print(self.cc)
C(13)()

13


In [34]:
dir(C)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

对一个对象定义call函数后，可以调用它。

In [35]:
from functools import wraps

class logit(object):
    def __init__(self, logfile='out.log'):
        self.logfile = logfile

    def __call__(self, func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # 打开logfile并写入
            with open(self.logfile, 'a') as opened_file:
                # 现在将日志打到指定的文件
                opened_file.write(log_string + '\n')
            # 现在，发送一个通知
            self.notify()
            return func(*args, **kwargs)
        return wrapped_function

    def notify(self):
        # logit只打日志，不做别的
        pass

装饰器是一个以函数为主要参数的可调用对象,在不影响函数内部运算的情况下，对函数作出修饰功能

In [36]:
@logit() # logit 类构造函数，构造后的类对象本身可被调用，且调用函数接受func为参数。
def myfunc1():
    pass

### 子类

In [41]:
class email_logit(logit): # 括号中是继承的对象
    '''
    一个logit的实现版本，可以在函数调用时发送email给管理员
    '''
    def __init__(self, email='admin@myproject.com', *args, **kwargs):
        self.email = email
        # super(email_logit, self).__init__(*args, **kwargs)
        super().__init__(*args,**kwargs)

    def notify(self):
        # 发送一封email到self.email
        print("email sent to ", self.email)
        return

In [44]:
@email_logit() # logit 类构造函数，构造后的类对象本身可被调用，且调用函数接受func为参数。
def myfunc1():
    return
myfunc1()


myfunc1 was called
email sent to  admin@myproject.com
